In [1]:
import pandas as pd
root_path = "/Users/ryanlindbeck/Development/resai/data"
roster_path = f"{root_path}/fanduel/roster-nba-3.csv"
df = pd.read_csv(roster_path)
print(df.head())
print(df.columns)

# https://www.rotowire.com/daily/nba/optimizer.php?site=FanDuel

          version https://git-lfs.github.com/spec/v1
0  oid sha256:0e2447034c4e57fd18f4ecd233232ad6773...
1                                         size 33662
Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')


In [2]:
# Assuming 'df' is your original DataFrame
non_injured_roster_df = df[df['Injury Indicator'].isna()]

# Initial setup
top_player_ids = ['96771-9646', '96771-110310']
salary_cap = 60000

position_requirements = {
    'PG': 2, 'SG': 2, 'SF': 2, 'PF': 2, 'C': 1
}

# Custom sorting function based on position requirements
def position_sort_key(pos):
    order = ['PG', 'SG', 'SF', 'PF', 'C']
    return order.index(pos)

# Select top players
top_players = non_injured_roster_df[non_injured_roster_df['Id'].isin(top_player_ids)]

# Initialize position counts based on top players
position_counts = {position: 0 for position in position_requirements}

# Assign positions to top players and update position counts
top_players['Chosen Position'] = top_players['Position'].apply(lambda x: x.split('/')[0])
for pos in top_players['Chosen Position']:
    position_counts[pos] += 1

# Calculate remaining salary
total_top_players_salary = top_players['Salary'].sum()
remaining_salary_cap = salary_cap - total_top_players_salary

# Initialize the final roster with top players
final_roster = top_players.copy()

# Remaining roster spots
remaining_roster_spots = sum(position_requirements.values()) - len(final_roster)

# Fill remaining positions
while remaining_roster_spots > 0:
    max_salary_per_player = remaining_salary_cap / remaining_roster_spots
    available_players = non_injured_roster_df[(non_injured_roster_df['Salary'] <= max_salary_per_player) & 
                                              (~non_injured_roster_df['Id'].isin(final_roster['Id']))]

    # Break if no available players
    if available_players.empty:
        break

    # Select the best player based on position need and FPPG
    best_player = None
    best_player_score = -1
    for _, player in available_players.iterrows():
        for pos in player['Position'].split('/'):
            if position_counts[pos] < position_requirements[pos]:
                player_score = player['FPPG'] / (position_requirements[pos] - position_counts[pos])
                if player_score > best_player_score:
                    best_player_score = player_score
                    best_player = player
                    chosen_position = pos

    if best_player is not None:
        best_player_df = pd.DataFrame([best_player])
        best_player_df['Chosen Position'] = chosen_position
        final_roster = pd.concat([final_roster, best_player_df], ignore_index=True)
        position_counts[chosen_position] += 1
        remaining_salary_cap -= best_player['Salary']
        remaining_roster_spots -= 1

# Calculate total salary
total_salary_final = final_roster['Salary'].sum()
remaining_salary_final = salary_cap - total_salary_final
total_fppg_final = final_roster['FPPG'].sum()

# Sort the final roster by position in the order of position requirements
final_roster_sorted = final_roster.sort_values(by='Chosen Position', key=lambda x: x.map(position_sort_key))

# Display final sorted roster
final_player_list_sorted = final_roster_sorted[['Nickname', 'Chosen Position', 'FPPG', 'Salary']]
print(final_player_list_sorted)
print(f"Total Salary: {total_salary_final}")
print(f"Remaining Salary: {remaining_salary_final}")
print(f"Total FPPG: {total_fppg_final}")

KeyError: 'Injury Indicator'